In [66]:
%matplotlib inline
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as f
import torch.nn as nn

# General approach

The goal is to find a maximum-weight bipartite matching where the cost function is unknown and depends on features of the inputs. Additionally, we might want a matching that trades off optimality and diversity by way of entropy regularization -- conveniently, this is what Sinkhorn does.

We have elements from the two sides $i, j$ with features $u_i, v_j$, the true cost is some $C^*_{ij}$, and we want to learn our own $C_{ij} = f(u_i, v_j; \theta)$. In one interesting case many of the $C^*_{ij}$ may be 0, meaning these edges do not exist and should not be matched.

Two approaches can be taken:

1. Two-stage approach

The two-stage approach is simple. One simply learns $f$ to predict the true observed $C^*_{ij}$ as well as possible, and then make matches based on the predictions of $f$. This approach should work well if $f$ can successfully learn the correct matching based on the features. There are some experimental details here -- which information exactly does $f$ get to observe -- do we get to see pairs of unmatched nodes, or only the actual matches made? $f$ here can be any learner.

2. Predict-and-optimize approach

The idea here is to still have $f$ output cost functions, but then pass these into a differentiable surrogate for the matching (Sinkhorn), and maximize the total weight of the observed matching. We expect this approach to win if $f$ cannot make perfect predictions. $f$ here must be a gradient based approach, i.e. a (shallow) NN.

# Some notes

One advantage of this compared to the Wilder et al paper is that for a problem this simple, Sinkhorn is much nicer to deal with than their Gurobi nightmare or even qpth, and can stay on the GPU.

## first steps

Consider the case where we either have edges or don't, and want to predict that from features only, in order to choose a maximum-weight matching.

In [39]:
dim = 10
N = 100
u_feats = torch.rand(N, dim)
v_feats = torch.rand(N, dim)

In [64]:
scores_mat = u_feats @ (v_feats.t())

In [65]:
edge_mat = (scores_mat > 2.5).float()

### training, batching, etc.

first just train on dataset to get architecture up

In [68]:
model = nn.Sequential(*[nn.Linear(dim*2, 128), nn.ReLU(), nn.Linear(128, 128), nn.ReLU(), nn.Linear(128, 1)])

Sequential(
  (0): Linear(in_features=20, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=1, bias=True)
)

### future notes

eventually we will need a case (for entropic matching) where many of the inputs are indistinguishable -- maybe just discrete and/or binary input features.

should try hiding some of the true features

eventually will need a more interesting function to predict edges